In [15]:
from __future__ import absolute_import, division, print_function, unicode_literals
import backtrader as bt
import datetime as dt
import pandas as pd

In [17]:
# # TODO: load data
price_raw = pd.read_parquet('./data/set')

In [18]:
price_raw.head(3)

,ticker,close,dividends,high,low,open,stock splits,volume
date,,,,,,,,
2019-11-13,ACE,4.549682,0.0,4.951701,4.392796,4.412407,0.0,1.188754e+09
2019-11-14,ACE,4.196689,0.0,4.647735,4.157467,4.588903,0.0,2.193090e+08
2019-11-15,ACE,4.118246,0.0,4.275132,3.902529,4.255521,0.0,1.387707e+08


In [49]:
# # TODO: Create a Strategy
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # TODO: plotting results
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25).subplot = True
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0]).plot = False

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [50]:
# # * select ticker
ticker = 'KTC'
cols = ['open', 'high', 'low', 'close']
ticker_df = price_raw[price_raw['ticker'] == ticker][cols]
data = bt.feeds.PandasData(dataname=ticker_df)

In [52]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set investing amount
    cerebro.broker.setcash(1e6)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.plot()

Starting Portfolio Value: 1000000.00
2002-12-12, Close, 0.47
2002-12-13, Close, 0.45
2002-12-16, Close, 0.45
2002-12-17, Close, 0.45
2002-12-18, Close, 0.44
2002-12-19, Close, 0.43
2002-12-20, Close, 0.43
2002-12-23, Close, 0.43
2002-12-24, Close, 0.42
2002-12-25, Close, 0.42
2002-12-26, Close, 0.42
2002-12-27, Close, 0.42
2002-12-30, Close, 0.42
2002-12-31, Close, 0.42
2003-01-01, Close, 0.42
2003-01-02, Close, 0.40
2003-01-03, Close, 0.41
2003-01-06, Close, 0.44
2003-01-06, BUY CREATE, 0.44
2003-01-07, BUY EXECUTED, Price: 0.45, Cost: 45.16, Comm 0.05
2003-01-07, Close, 0.44
2003-01-08, Close, 0.43
2003-01-09, Close, 0.43
2003-01-10, Close, 0.42
2003-01-10, SELL CREATE, 0.42
2003-01-13, SELL EXECUTED, Price: 0.42, Cost: 45.16, Comm 0.04
2003-01-13, OPERATION PROFIT, GROSS -2.90, NET -2.99
2003-01-13, Close, 0.43
2003-01-13, BUY CREATE, 0.43
2003-01-14, BUY EXECUTED, Price: 0.43, Cost: 42.90, Comm 0.04
2003-01-14, Close, 0.44
2003-01-15, Close, 0.46
2003-01-16, Close, 0.50
2003-01-17,

ValueError: Axis limits cannot be NaN or Inf